# Legionella


Neste trabalho...


## Análise da sequência e das features presentes no NCBI

Começamos por aceder ao NCBI para fazer download de um ficheiro no formato __genbank__ com informação correspondente à nossa zona do genoma, da posição 270001 a 505535.

O primeiro passo foi descobrir qual o __genbank_id__ associado ao accession __NC_002942.5__.
Para isto usamos a função `Entrez.search`. 

Após saber o __genbank_id__, obtivemos o __genbank__ usando a função `Entrez.efetch`.

```python
def fetch_genbank(start, end):
    """
    Procura no NCBI pelo Accession NC_002942.5 e 
    faz download de toda a informação em formato genbank,
    retornando o respectivo record com as features
    entre a posição start e end.
    """

    Entrez.email = "*@*.*"

    handle = Entrez.esearch(db="nucleotide", term="NC_002942.5")
    record = Entrez.read(handle)
    genbank_id = record["IdList"][0]
    handle.close()

    handle = Entrez.efetch(db="nucleotide",
                           rettype="gbwithparts",
                           retmod="text",
                           id=genbank_id,
                           seq_start=start,
                           seq_stop=end)
    record = SeqIO.read(handle, "gb")
    handle.close()

    return record
```

Esta função foi adicionada ao módulo `util.www`.


Deste `record` extraímos as features __gene__, __CDS__, __tRNA__ e __rRNA__.

```python
def extract_features(record):
    """
    Das várias features encontradas no ficheiro genbank, extraímos:
      - gene
      - CDS
      - tRNA
      - rRNA

    Esta função retorna uma lista apenas com as features
    dos tipos indicados acima.
    """

    result = []
    features = record.features
    feature_count = len(features)
    types = ["gene", "CDS", "tRNA", "rRNA"]

    for i in range(feature_count):
        feature = features[i]
        should_extract = feature.type in types

        if should_extract:
            result.append(feature)

    return result
```


Para finalizar, convertemos esta lista de `record` para um dicionário.
A cada um dos `record` retirámos:
- db_xref
- EC_number
- function
- gene
- gene_synonym
- locus_tag
- note
- product
- protein_id
- translation

Além disso, também ficamos com informação sobre a localização de cada gene e em qual das cadeias está presente.

```python
def features_to_dictionary(start, features):
    """
    A cada uma das features, extraimos a seguintes propriedades:
      - db_xref
      - EC_number
      - function
      - gene
      - gene_synonym
      - locus_tag
      - note
      - product
      - protein_id
      - translation

    Também extraímos a localização:
      - start
      - end
      - strand
    """
    dictionary = {}
    properties = ["db_xref",
                  "EC_number",
                  "function",
                  "gene",
                  "gene_synonym",
                  "note",
                  "product",
                  "protein_id",
                  "translation"]

    for feature in features:
        tag = feature.qualifiers["locus_tag"][0]

        if not tag in dictionary:
            # se esta tag ainda não existe,
            # criar um dicionário vazio para ela
            dictionary[tag] = {}

        location = feature.location
        dictionary[tag]["location"] = {}
        dictionary[tag]["location"]["start"] = location._start + start
        dictionary[tag]["location"]["end"] = location._end + start - 1
        dictionary[tag]["location"]["strand"] = location._strand

        if feature.type in ["tRNA", "rRNA"]:
            # Taggar as features com estes dois tipos
            dictionary[tag]["type"] = feature.type
        else:
            # se não, taggar com "mRNA"
            dictionary[tag]["type"] = "mRNA"

        for prop in properties:
            if prop in feature.qualifiers:
                value = feature.qualifiers[prop][0]

                if prop in dictionary[tag]:
                    # se já encontramos esta propriedade
                    # para este locus_tag
                    # verificar que é a mesma
                    current_value = dictionary[tag][prop]
                    assert value == current_value
                else:
                    # se não, adicionar
                    dictionary[tag][prop] = value

    return dictionary
```

No final:

```python
import util.www as www

start = 270001
end = 505535

record = www.fetch_genbank(start, end)
features = extract_features(record)
dictionary = features_to_dictionary(start, features)
print(dictionary)
```

```json
{
  "lpg0232": {
    "db_xref": "GeneID:19831799",
    "function": "Transcription factors / DNA binding proteins",
    "gene": "np20",
    "location": {
      "end": 270569,
      "start": 270036,
      "strand": 1
    },
    "product": "transcriptional regulator np20",
    "protein_id": "YP_094286.1",
    "translation": "MIGCCLIIFPC...",
    "type": "mRNA"
  },
  "lpg0233": {
    "EC_number": "4.1.1.7",
    "db_xref": "GeneID:19831800",
    "function": "Biodegradation of Xenobiotics",
    "gene": "mdlC",
    "location": {
      "end": 272278,
      "start": 270686,
      "strand": -1
    },
    "product": "benzoylformate decarboxylase",
    "protein_id": "YP_094287.1",
    "translation": "MKKTGSDVLK...",
    "type": "mRNA"
  },
  // ...
}
```

In [3]:
import util.rw as rw
import pandas as pd

pd.options.display.max_rows = 1000

ncbi = rw.read_json(".ncbi.json")
df = pd.DataFrame(ncbi).transpose()
df = df.fillna("")
df[["EC_number", "db_xref", "function", "gene", "product", "protein_id", "type"]]

EC_number          db_xref  \
lpg0232              GeneID:19831799   
lpg0233     4.1.1.7  GeneID:19831800   
lpg0234              GeneID:19831801   
lpg0235              GeneID:19831802   
lpg0236              GeneID:19831803   
lpg0237              GeneID:19831804   
lpg0238     1.2.1.8  GeneID:19831805   
lpg0239    2.6.1.19  GeneID:19831806   
lpg0240              GeneID:19831807   
lpg0241     3.5.1.2  GeneID:19831808   
lpg0242    1.1.1.95  GeneID:19831809   
lpg0243              GeneID:19831810   
lpg0244    1.16.1.1  GeneID:19831811   
lpg0245              GeneID:19831812   
lpg0246              GeneID:19831813   
lpg0247              GeneID:19831814   
lpg0248              GeneID:19831815   
lpg0249   1.14.19.1  GeneID:19831816   
lpg0250              GeneID:19831817   
lpg0251              GeneID:19831818   
lpg0252              GeneID:19831819   
lpg0253              GeneID:19831820   
lpg0254              GeneID:19831821   
lpg0255              GeneID:19831822   
lpg0256              GeneID:19831823   
lpg0257              GeneID:19831824   
lpg0258              GeneID:19831825   
lpg0259              GeneID:19831826   
lpg0260              GeneID:19831827   
lpg0261              GeneID:19831828   
lpg0262     2.7.7.7  GeneID:19831829   
lpg0263              GeneID:19831830   
lpg0264              GeneID:19831831   
lpg0265              GeneID:19831832   
lpg0266              GeneID:19831833   
lpg0267              GeneID:19831834   
lpg0268              GeneID:19831835   
lpg0269              GeneID:19831836   
lpg0270    2.4.2.11  GeneID:19831837   
lpg0271              GeneID:19831838   
lpg0272              GeneID:19831839   
lpg0273              GeneID:19831840   
lpg0274              GeneID:19831841   
lpg0275              GeneID:19831842   
lpg0276              GeneID:19831843   
lpg0277              GeneID:19831844   
lpg0278     2.7.3.-  GeneID:19831845   
lpg0279              GeneID:19831846   
lpg0280              GeneID:19831847   
lpg0281              GeneID:19831848   
lpg0282              GeneID:19831849   
lpg0283     1.2.2.1  GeneID:19831850   
lpg0284              GeneID:19831851   
lpg0285              GeneID:19831852   
lpg0286     1.-.-.-  GeneID:19831853   
lpg0287              GeneID:19831854   
lpg0288              GeneID:19831855   
lpg0289     2.7.4.1  GeneID:19831856   
lpg0290              GeneID:19831857   
lpg0291              GeneID:19831858   
lpg0292              GeneID:19831859   
lpg0293    1.3.99.-  GeneID:19831860   
lpg0294              GeneID:19831861   
lpg0295              GeneID:19831862   
lpg0296              GeneID:19831863   
lpg0297              GeneID:19831864   
lpg0298     5.2.1.8  GeneID:19831865   
lpg0299   1.1.1.262  GeneID:19831866   
lpg0300     1.5.1.3  GeneID:19831867   
lpg0301              GeneID:19831868   
lpg0302              GeneID:19831869   
lpg0303              GeneID:19831870   
lpg0304              GeneID:19831871   
lpg0305              GeneID:19831872   
lpg0306              GeneID:19831873   
lpg0307              GeneID:19831874   
lpg0309              GeneID:19831875   
lpg0310              GeneID:19831876   
lpg0311              GeneID:19831877   
lpg0312              GeneID:19831878   
lpg0313              GeneID:19831879   
lpg0314     3.6.5.3  GeneID:19831880   
lpg0315              GeneID:19831881   
lpg0316              GeneID:19831882   
lpg0317              GeneID:19831883   
lpg0318              GeneID:19831884   
lpg0319              GeneID:19831885   
lpg0320              GeneID:19831886   
lpg0321              GeneID:19831887   
lpg0322     2.7.7.6  GeneID:19831888   
lpg0323     2.7.7.6  GeneID:19831889   
lpg0324              GeneID:19831890   
lpg0325              GeneID:19831891   
lpg0326     3.6.5.3  GeneID:19831892   
lpg0327     3.6.5.3  GeneID:19831893   
lpg0328              GeneID:19831894   
lpg0329              GeneID:19831895   
lpg0330              GeneID:19831896   
lpg0331              GeneID:19831897   
lpg0332   